## 4.1画像データ

In [1]:
from PIL import Image
import numpy as np
import torch

In [2]:
PATH = "../deep-learning-with-pytorch-ja-main/data/p1ch2/bobby.jpg"
img = Image.open(PATH)

In [3]:
img_arr = np.array(img)
print(img_arr.shape)

(720, 1280, 3)


In [20]:
#permuteは次元の位置を変更する

#tensorflowはチャネルが4
#torchは2
#batchはどちらも最初
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
print(img.shape, out.shape)

torch.Size([720, 1280, 3]) torch.Size([3, 720, 1280])


In [7]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
#カメラの写真のフォーマット8ビットのint型

In [8]:
import os
data_dir = "../deep-learning-with-pytorch-ja-main/data/p1ch4/image-cats"
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] =='.png']

#os.path.splitext(ファイル名)で拡張子を取得

In [9]:
#batchに画像データ三つを上書き
for i, filename in enumerate(filenames):
    img_arr = Image.open(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(np.array(img_arr))
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

In [40]:
batch.shape, batch[2][0][0][0]

(torch.Size([3, 3, 256, 256]), tensor(238, dtype=torch.uint8))

In [41]:
#データの正規化

#ニューラルネットでは0から1、または-1から1の範囲で性能が高い
#画像データは要素を浮動小数点にして、正規化するのが普通

#8bit符号なし表現なら最大値の255で割る手法がある
batch = batch.float()
batch /= 255.0

In [48]:
batch.shape[1], batch.shape

(3, torch.Size([3, 3, 256, 256]))

In [58]:
batch.shape, batch[:,1].shape, batch[:,1,1].shape

(torch.Size([3, 3, 256, 256]), torch.Size([3, 256, 256]), torch.Size([3, 256]))

In [61]:
batch[:,1,:,:][0][0][0], batch[:,1][0][0][0]

(tensor(0.4632), tensor(0.4632))

In [47]:
#もう一つの正規化手法は平均0、標準偏差1にする標準化
n_chanels = batch.shape[1]
for c in range(n_chanels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean ) / std

In [62]:
#三次元データ（CT)
#次元はバッチ、チャネル、深さ、たて、横の五次元になる

In [16]:
import imageio
dir_path = "../deep-learning-with-pytorch-ja-main/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083"
vol_arr = imageio.volread(dir_path, "DICOM")
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%69/99 files (69.7%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 69/99  (69.799/99  (100.0%)


(99, 512, 512)

In [17]:
vol = torch.from_numpy(vol_arr).float()
vol.shape

torch.Size([99, 512, 512])

In [18]:
#バッチ方向がない四次元
vol = torch.unsqueeze(vol, 0)
vol.shape

torch.Size([1, 99, 512, 512])

In [19]:
#表形式のデータ

In [20]:
import csv

In [24]:
wine_path = "../deep-learning-with-pytorch-ja-main/data/p1ch4/tabular-wine/winequality-white.csv"
#1行目はカラム名のためスキップして読み込み
wine_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
wine_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [23]:
col_list = next(csv.reader(open(wine_path), delimiter=";"))
wine_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [25]:
wineq = torch.from_numpy(wine_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [26]:
#連続値、順序尺度、カテゴリカル変数には注意

In [27]:
#スコアの表現

#スコアはラベルとして扱う分類タスクか、連続値として回帰タスクか

In [28]:
#最後の列が目的変数のためそれを除く
data = wineq[:, :-1]
print(data.shape)

target = wineq[:, -1]
print(target.shape)

torch.Size([4898, 11])
torch.Size([4898])


In [30]:
target.dtype

torch.float32

In [33]:
#ターゲットがワインの色名のような文字列のラベルであれば、文字列を整数値にして同様にアプローチできる
target = wineq[:, -1].long()
target.dtype

torch.int64

In [34]:
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [52]:
#ワンホットエンコーディング
target_onehot = torch.zeros(target.shape[0], 10)
print(target_onehot.shape, target.shape, target.unsqueeze(1).shape)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)[19]

torch.Size([4898, 10]) torch.Size([4898]) torch.Size([4898, 1])


tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [ ]:
#scatter_で最後が_で終わるのは内部でテンソルを上書きする
#引数は
#one_hotにする次元 今回は二つ目次元の10側が目的
#one_hotにしたい列（ラベル）　このindexはone_ohtで表現するテンソルと同じ次元にする
#入力させる値

In [53]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [58]:
#テンソルの計算
print(data.shape)
data_mean = torch.mean(data, dim=0)
data_mean.shape

torch.Size([4898, 11])


torch.Size([11])

In [59]:
data_var = torch.var(data, dim=0)
data_var.shape

torch.Size([11])

In [62]:
#正規化（標準化）
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized[1], data[1]

(tensor([-0.6574,  0.2159,  0.0480, -0.9447,  0.1477, -1.2529, -0.1497, -0.0091,
          0.7400,  0.0013, -0.8242]),
 tensor([6.3000e+00, 3.0000e-01, 3.4000e-01, 1.6000e+00, 4.9000e-02, 1.4000e+01,
         1.3200e+02, 9.9400e-01, 3.3000e+00, 4.9000e-01, 9.5000e+00]))

In [63]:
#閾値の設定
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [65]:
#bad_indexesの要素のうち20個がTure
#advanced indexing機能を使い、torch.boolのテンソルを利用し、 インデックスをつける

In [66]:
bad_data = data[bad_indexes]
bad_data.shape
#bad_dataテンソルは20行
#条件にTrueで返した行
#列は11列全て保有

torch.Size([20, 11])

In [68]:
#上の方法を使いワインの状態を三段階に分ける
bad_data = data[target <= 3]
mid_data = data[(target >3) & ( target <= 7)]
good_data = data[target >= 7]

